In [8]:
!uv pip install evaluate -q

import tensorflow_datasets as tfds
import torch
import numpy as np
from datasets import Dataset, Audio
from transformers import (
    Wav2Vec2Processor, 
    Wav2Vec2ForSequenceClassification, 
    TrainingArguments, 
    Trainer
)
import evaluate
import IPython.display as ipd



import torch
import numpy as np
import evaluate
from datasets import load_from_disk
from transformers import (
    TrainingArguments,
    Trainer,
    Wav2Vec2Processor, 
    Wav2Vec2ForSequenceClassification,
    Wav2Vec2FeatureExtractor
)


import os
from transformers.trainer_utils import get_last_checkpoint

import warnings
warnings.filterwarnings("ignore")

In [9]:
ds_data, ds_info = tfds.load(
    "crema_d",
    with_info=True,
    as_supervised=False,
    split=['train', 'validation', 'test']
)



# Extract label mappings (0=Neutral, 1=Happy, etc.) from TFDS metadata
label_names = ds_info.features['label'].names
label2id = {label: i for i, label in enumerate(label_names)}
id2label = {i: label for i, label in enumerate(label_names)}
print(f"Labels found: {label_names}")

# --- 2. Bridge: Convert TFDS to Hugging Face Dataset ---
# Wav2Vec2 Trainer works best with Hugging Face Datasets. 
# Since CREMA-D is small (~2GB), we can convert it in memory.

def tfds_to_hf_dataset(tf_dataset):
    data_dict = {"audio": [], "label": []}
    for sample in tf_dataset:
        audio = sample['audio'].numpy()
        label = sample['label'].numpy()
        
        # Normalize audio if it's integer PCM (Wav2Vec2 expects float inputs)
        # CREMA-D in TFDS is often int64; we convert to float32
        audio = audio.astype(np.float32)
        if np.abs(audio).max() > 1.0:
            audio = audio / 32768.0  # Normalize 16-bit PCM to [-1, 1]
            
        data_dict["audio"].append(audio)
        data_dict["label"].append(label)
    
    return Dataset.from_dict(data_dict)

print("Converting TFDS to Hugging Face format... (this may take a minute)")
train_dataset = tfds_to_hf_dataset(ds_data[0])
eval_dataset = tfds_to_hf_dataset(ds_data[1])
test_dataset = tfds_to_hf_dataset(ds_data[2])


print("Saving converted dataset to disk...")
train_dataset.save_to_disk("./crema_hf/train")
eval_dataset.save_to_disk("./crema_hf/eval")
test_dataset.save_to_disk("./crema_hf/test")

print("Saved! You can now load it later using from_disk()")

Labels found: ['NEU', 'HAP', 'SAD', 'ANG', 'FEA', 'DIS']
Converting TFDS to Hugging Face format... (this may take a minute)
Saving converted dataset to disk...


Saving the dataset (0/2 shards):   0%|          | 0/5144 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/738 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1556 [00:00<?, ? examples/s]

Saved! You can now load it later using from_disk()


In [12]:
# -----------------------------
# 1️⃣ Load datasets
# -----------------------------
print("Loading dataset from disk...")
train_dataset = load_from_disk("./crema_hf/train")
eval_dataset  = load_from_disk("./crema_hf/eval")
test_dataset  = load_from_disk("./crema_hf/test")

# -----------------------------
# 2️⃣ Label configuration
# -----------------------------
label_names = ["NEU", "HAP", "SAD", "ANG", "FEA", "DIS"]
label2id = {label: i for i, label in enumerate(label_names)}
id2label = {i: label for i, label in enumerate(label_names)}
print(f"Labels configured: {label_names}")

# -----------------------------
# 3️⃣ Processor & Model
# -----------------------------
model_id = "facebook/wav2vec2-base"
processor = Wav2Vec2FeatureExtractor.from_pretrained(model_id)
target_sampling_rate = processor.sampling_rate

model = Wav2Vec2ForSequenceClassification.from_pretrained(
    model_id,
    num_labels=len(label_names),
    label2id=label2id,
    id2label=id2label
)
model.freeze_feature_extractor()  # freeze CNN feature extractor

# -----------------------------
# 4️⃣ Preprocessing
# -----------------------------
def preprocess_function(examples):
    audio_arrays = examples["audio"]
    return processor(
        audio_arrays,
        sampling_rate=target_sampling_rate,
        max_length=target_sampling_rate * 5,
        truncation=True,
        padding="max_length",
        return_tensors="pt"
    )

print("Preprocessing datasets...")

encoded_train = train_dataset.map(preprocess_function, batched=True)
encoded_eval  = eval_dataset.map(preprocess_function, batched=True)
encoded_test  = test_dataset.map(preprocess_function, batched=True)

# Rename label column properly (assign back)
encoded_train = encoded_train.rename_column("label", "labels")
encoded_eval  = encoded_eval.rename_column("label", "labels")
encoded_test  = encoded_test.rename_column("label", "labels")

# Set PyTorch format (in-place)
columns = ["input_values", "labels"]
encoded_train.set_format(type="torch", columns=columns)
encoded_eval.set_format(type="torch", columns=columns)
encoded_test.set_format(type="torch", columns=columns)


accuracy = evaluate.load("accuracy")
def compute_metrics(eval_pred):
    preds = np.argmax(eval_pred.predictions, axis=1)
    return accuracy.compute(predictions=preds, references=eval_pred.label_ids)



training_args = TrainingArguments(
    output_dir="./wav2vec2-base-emotion-model",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=2,
    eval_strategy="epoch",
    save_strategy="epoch",
    num_train_epochs=20,
    learning_rate=3e-5,
    fp16=True,
    logging_steps=10,
    load_best_model_at_end=True,
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_train,
    eval_dataset=encoded_eval,
    tokenizer=processor,  # feature extractor acts as tokenizer
    compute_metrics=compute_metrics,
)



if torch.cuda.is_available():
    print(f"Training on GPU: {torch.cuda.get_device_name(0)}")
else:
    print("WARNING: GPU not detected.")



output_dir = "./wav2vec2-base-emotion-model"
last_checkpoint = get_last_checkpoint(output_dir)

if last_checkpoint is not None:
    print(f"Resuming training from {last_checkpoint}")
    trainer.train(resume_from_checkpoint=last_checkpoint)
else:
    print("No checkpoint found, training from scratch")
    trainer.train()


trainer.save_model(os.path.join(output_dir, "final_model"))
processor.save_pretrained(os.path.join(output_dir, "final_processor"))


print("Evaluating on test set...")
test_results = trainer.predict(encoded_test)
print(f"Test Accuracy: {test_results.metrics['test_accuracy']:.4f}")



with open(os.path.join(output_dir, "test_metrics.json"), "w") as f:
    json.dump(test_results.metrics, f, indent=4)

Loading dataset from disk...
Labels configured: ['NEU', 'HAP', 'SAD', 'ANG', 'FEA', 'DIS']


Some weights of Wav2Vec2ForSequenceClassification were not initialized from the model checkpoint at facebook/wav2vec2-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'projector.bias', 'projector.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Preprocessing datasets...


Training on GPU: Tesla T4
No checkpoint found, training from scratch


Epoch,Training Loss,Validation Loss,Accuracy
1,1.297300,1.105924,0.630081
2,0.940300,0.885753,0.689702
3,0.708500,0.792240,0.726287
4,0.527100,0.756590,0.741192
5,0.541500,0.871263,0.727642
6,0.467800,0.794341,0.761518
7,0.452100,0.866151,0.760163
8,0.245300,1.092163,0.726287
9,0.245000,0.891721,0.773713
10,0.206100,1.031553,0.761518


RuntimeError: [enforce fail at inline_container.cc:626] . unexpected pos 49603136 vs 49603024